<a href="https://colab.research.google.com/github/JangHanjun/Natural_Language_Programming/blob/main/Chapter11/CharRNNLotData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
sentences = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [7]:
char_set = list(set(sentences))
char_dic = {c : i for i, c in enumerate(char_set)}

print(char_dic)

{'s': 0, 'i': 1, 't': 2, 'm': 3, ',': 4, "'": 5, 'b': 6, 'f': 7, 'l': 8, ' ': 9, 'a': 10, 'e': 11, 'p': 12, 'c': 13, 'd': 14, 'k': 15, 'w': 16, 'g': 17, 'h': 18, 'u': 19, 'r': 20, 'y': 21, '.': 22, 'n': 23, 'o': 24}


In [8]:
dic_size = len(char_dic)

In [9]:
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

In [ ]:
x_data = []
y_data = []

for i in range(0, len(sentences) - sequence_length):
  x_str = sentences[i:i+sequence_length]
  y_str = sentences[i+1:i+sequence_length+1]

  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

In [12]:
print(x_data[0])
print(y_data[0])

[1, 7, 9, 21, 24, 19, 9, 16, 10, 23]
[7, 9, 21, 24, 19, 9, 16, 10, 23, 2]


In [17]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print(X.shape)
print(Y.shape)

torch.Size([170, 10, 25])
torch.Size([170, 10])


In [ ]:
print(X[0])

In [22]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, hidden_dim,bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [23]:
net = Net(dic_size, hidden_size, 2)  # 층이 2개

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [25]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [26]:
print(outputs.view(-1, dic_size).shape)

torch.Size([1700, 25])


In [27]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [29]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  # result.shape = [170, 10]
  results = outputs.argmax(dim=2)
  predict_str = ""
  for j, result in enumerate(results):
    if j == 0:
      predict_str += ''.join([char_set[t] for t in result])
    else:
      predict_str += char_set[result[-1]]
  print(predict_str)

korrroorkkrrrrrroororkrorroooorkorrorooroororooorrrrrrrrrrrrrrooorrorrrroorkkoroooorrkrrrokrrrrrrrrrrrrkkorroororroorkrrrrrrrrkorrrrrrrrrrrrkrroorrrrrrrkorrrrrroorkrrrrrroorrrrrrk
 mmo m mm mmmmo m m m m  mmmm m mmt mo m om   o mmmmm mt mommommo mmmoo  mm mmm m mmt m  m mm  mommo mommmmo m mo m mmmmo mommmm  m  mo mo  mmmo m m  m mmmo   mmm  mmmmo mom  mo m
t e   e e             e                      e                                  e t   e e          e              e   e e          e                      e e                  e   
t ecy s .cy ycc  ccytry aryry yry yry acyyy yycyrr r y  rc rycy rcy rycyr tccrt yca yca acyyrccr ycy  ycy  ccct ycy tryry aryyyy  cyrrccyc rycy cct ycy yryrcyc cca cry ycycyrcy  c
t t s t s t             t   t     t s t s    s s                  s               t   t t s        s    t         s   s s t   s    s          t         t s       t       t t      
  e   t    ee ee    ee        e e      e e  e   e e  ee   e   ee   e e   e    e     o      eee  e e 